In [1]:
import numpy as np
import gym
import random

In [2]:
env = gym.make("FrozenLake-v0")

In [3]:
action_size = env.action_space.n
state_size = env.observation_space.n

In [4]:
# Create our Q table with state_size rows and action_size columns (64x4)
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


#create hyper-parameters


In [12]:
total_episodes = 30000       # Total episodes
learning_rate = 0.7          # Learning rate
max_steps = 99               # Max steps per episode
gamma = 0.95                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.005            # Exponential decay rate for exploration prob

In [13]:
from tqdm import tqdm

# List of rewards
rewards = []

# 2 For life or until learning is stopped
for episode in tqdm(range(total_episodes)):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])
            #print(exp_exp_tradeoff, "action", action)

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
            #print("action random", action)
            
        
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)
    

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)

100%|██████████| 30000/30000 [00:46<00:00, 644.15it/s]

Score over time: 0.5020333333333333
[[3.35995692e-01 6.85312929e-02 4.84371168e-02 8.76328308e-02]
 [1.93292488e-02 6.94751993e-03 6.56083040e-03 2.86630774e-01]
 [1.33686145e-02 3.44560181e-02 2.32055054e-02 1.57297142e-01]
 [1.51907729e-02 2.19639425e-03 5.50681239e-03 4.17108707e-02]
 [4.40091677e-01 3.17389799e-01 3.56037614e-03 4.15271997e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.10786427e-04 2.59460865e-05 1.22351438e-01 5.98695964e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.41552272e-01 6.83545799e-02 2.73747781e-02 4.53743148e-01]
 [7.10780506e-02 6.72202856e-01 3.29372379e-02 9.41063084e-03]
 [2.72303233e-01 1.61687104e-03 1.29566574e-02 6.79906388e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.44080036e-01 9.79322685e-02 9.29958961e-01 1.12178573e-01]
 [3.56200789e-01 9.99844266e-01 1.49496373e-01 2.02158297e-01]
 [0.00000000e+00 0.

In [14]:
ACTION_LOOKUP = {0: 'LEFT', 1: 'DOWN', 2: 'RIGHT', 3: 'UP'}

In [15]:
q_optimal = [ACTION_LOOKUP[np.argmax(q)] for q in qtable]
q_optimal = np.reshape(q_optimal, (4,4))
print(q_optimal)

[['LEFT' 'UP' 'UP' 'UP']
 ['LEFT' 'LEFT' 'RIGHT' 'LEFT']
 ['UP' 'DOWN' 'LEFT' 'LEFT']
 ['LEFT' 'RIGHT' 'DOWN' 'LEFT']]


#use q table to play frozen lake

In [16]:
env.reset()

for episode in range(5):
    state = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            if new_state == 15:
                print("We reached our Goal 🏆")
            else:
                print("We fell into a hole ☠️")
            
            # We print the number of step it took.
            print("Number of steps", step)
            
            break
        state = new_state
env.close()

****************************************************
EPISODE  0
****************************************************
EPISODE  1
  (Down)
SFFF
FHFH
FFFH
HFFG
We reached our Goal 🏆
Number of steps 34
****************************************************
EPISODE  2
  (Down)
SFFF
FHFH
FFFH
HFFG
We reached our Goal 🏆
Number of steps 6
****************************************************
EPISODE  3
****************************************************
EPISODE  4
  (Down)
SFFF
FHFH
FFFH
HFFG
We reached our Goal 🏆
Number of steps 76


In [17]:
q_optimal = [ACTION_LOOKUP[np.argmax(q)] for q in qtable]
q_optimal = np.reshape(q_optimal, (4,4))
print(q_optimal)

[['LEFT' 'UP' 'UP' 'UP']
 ['LEFT' 'LEFT' 'RIGHT' 'LEFT']
 ['UP' 'DOWN' 'LEFT' 'LEFT']
 ['LEFT' 'RIGHT' 'DOWN' 'LEFT']]
